In [1]:
from socket import *
import struct
import time
import sys

# Port number definitions
# (May have to be adjusted if they collide with ports in use by other programs/services.)
TCP_PORT = 12100

# Address to listen on when acting as server.
# The address '' means accept any connection for our 'receive' port from any network interface
# on this system (including 'localhost' loopback connection).
LISTEN_ON_INTERFACE = ''


In [2]:
def read_message(tcp_socket):
    new_msg = True
    fileCounter = 0
    #Accept the connection, creating a connection socket
    clientSocket, address = tcp_socket.accept()
    #Print the address and port of the sender   
    print(f"Connection from {address} has been established.")
    fileFlag = True
    while fileFlag:
        fileCounter += 1

        #reading in header for number of lines
        full_header = b''
        for i in range(4):
            header = clientSocket.recv(1)
            full_header += header   
        numLines = int.from_bytes(full_header, "big")
        fullMsg = ''
        if numLines <= 0:
            clientSocket.send(bytes('Q', 'utf-8'))
            clientSocket.close()
            tcp_socket.close()
            fileFlag = False
                
        elif numLines > 0:  
            numLinesRead = 0
            flag = True
            while flag:
                if numLinesRead >= numLines:
                    flag = False
                else:
                    fullLine = b''
                    current_byte = b'' 
                    while current_byte != b'\n':
                        
                        current_byte = clientSocket.recv(1)
                        fullLine += current_byte

                    fullMsg += fullLine.decode('utf-8')
                    
                    numLinesRead += 1
                    print(numLinesRead)
        
            clientSocket.send(bytes('A', 'utf-8'))
            
            print(fullMsg)
            print(f'File Counter: {fileCounter}')
            #write_to_file(fullMsg, fileCounter)
            
   
    

In [3]:
def write_to_file(msg, fileNumber): 
    file = open(f'File{fileNumber}.txt', "w")
    file.write(msg)
    file.close

In [4]:
def tcp_receive(listen_on, listen_port):
    """
    - Listen for a TCP connection on a designated "listening" port
    """
    _ADDRESS_ = (listen_on, listen_port)
    tcp_socket = socket(AF_INET, SOCK_STREAM)
    tcp_socket.bind(_ADDRESS_)
    tcp_socket.listen(5)    
    read_message(tcp_socket)       
        
        
    
    """
    - Repeat until a zero-length message is received:
      - Receive a message, saving it to a text-file (1.txt for first file, 2.txt for second file, etc.)
      - Send a single-character response 'A' to indicate that the upload was accepted.
    - Send a 'Q' to indicate a zero-length message was received.
    - Close data connection.

    :param int listen_port: Port number on the server to listen on
    """

    print('tcp_receive (server): listen_port={0}'.format(listen_port))


In [5]:
tcp_receive(LISTEN_ON_INTERFACE, TCP_PORT)

Connection from ('192.168.1.181', 58363) has been established.
1
2
3
Hello, World!
There are 3 lines
done

File Counter: 1
1
2
hello world
stuff

File Counter: 2
1
foo bar

File Counter: 3
tcp_receive (server): listen_port=12100
